In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
#from numba import jit, cuda

In [2]:
df_x = pd.read_pickle("../../BIO_Ml/Schizophrenia/one_by_one/mvals_train_val.pkl")
df_y = pd.read_pickle("../../BIO_Ml/Schizophrenia/one_by_one/pheno_train_val.pkl")["Status"]
print(df_y)

subject_id
GSM4599914          Control
GSM4599916          Control
GSM4599917    Schizophrenia
GSM4599919    Schizophrenia
GSM4599923    Schizophrenia
                  ...      
GSM2126304          Control
GSM2126543          Control
GSM2126657    Schizophrenia
GSM2126455          Control
GSM2126599    Schizophrenia
Name: Status, Length: 2015, dtype: object


In [3]:
# control = 0
df_y = LabelEncoder().fit_transform(df_y)
print(df_y)

[0 0 1 ... 1 0 1]


In [4]:
df_x["status"] = df_y
print(df_x.head())

            cg18147296  cg13938959  cg12445832  cg23999112  cg11527153  \
subject_id                                                               
GSM4599914    2.762321    0.843977   -0.016555    1.372435    3.005380   
GSM4599916    2.685985    1.294113   -0.249022    0.993145    2.790599   
GSM4599917    2.854838    1.527030   -0.310427    1.113818    3.004341   
GSM4599919    2.889056    1.682198    0.389953    1.793420    3.201766   
GSM4599923    2.943605    0.930647   -0.436929    1.068895    2.669822   

            cg04195702  cg08128007  cg23733394  cg13371836  cg04407431  ...  \
subject_id                                                              ...   
GSM4599914    2.615756    2.206671   -1.367863   -4.865131   -5.510768  ...   
GSM4599916    2.906080    2.203198   -1.799485   -4.631597   -5.251224  ...   
GSM4599917    2.778681    2.183322   -0.678120   -4.859093   -4.796586  ...   
GSM4599919    2.643427    2.676389    1.185422   -4.666803   -5.072614  ...   
GSM4599

In [5]:
df_control = df_x[(df_x['status'] == 0)]
df_schiz = df_x[(df_x['status'] == 1)]

In [6]:
from scipy.stats import ks_2samp

print(ks_2samp(df_control["cg13782905"], df_schiz["cg13782905"])[1])

0.917971526475926


### Отбор признаков по `p-value` в `k_test`.
#### Отбор `100`, `200`, `300` признаков для дальнейшего обучения модели.

In [ ]:
from tqdm.notebook import tqdm

df_len = df_control.shape[1]
print(df_len)

#@jit(target ="cuda")
def get_cpg_p_ktest(df_healthy, df_ill):
    cpg_p_values = []

    def get_p_gen(df_healthy, df_ill):
        for i in range(df_len-2): # -2 потому что в последнем элементе лежит status
            yield df_healthy.columns[i], ks_2samp(df_healthy.iloc[:, i], df_ill.iloc[:, i])[0]

    p_gen = get_p_gen(df_healthy, df_ill)
    for p in tqdm(p_gen, total=df_len):
        cpg_p_values.append(p)
    return cpg_p_values


cpg_p_values = get_cpg_p_ktest(df_control.drop('status', axis=1), df_schiz.drop('status', axis=1))

361391


  0%|          | 0/361391 [00:00<?, ?it/s]

#### Бекап списка для каждого cpg-сайта в формате `(cpg-site[0], p-value[0]), ... , (cpg-site[n-1], p-value[n-1])`

In [ ]:
import pickle

with open('feature_selection_results/cpg_p_values_unsorted.pkl', 'wb') as fp:
    pickle.dump(cpg_p_values, fp)

In [ ]:
#cpg_p_values = pickle.load(fp)

#### Сортировка, обрезка и сохранение списка cpg-сайтов

In [ ]:
def sort_slice(values, n):
    return sorted(values, key=lambda tup: tup[1], reverse=True)[:n]


top_100_cpgs_ktest = sort_slice(cpg_p_values, 100)


top_200_cpgs_ktest = sort_slice(cpg_p_values, 200)
top_300_cpgs_ktest = sort_slice(cpg_p_values, 300)
def write_results(list_to_write, n, form):
    with open(f"feature_selection_results/{form}_cpgs_{str(n)}.txt", 'w') as file:
        if len(list_to_write) == 0:
            file.write('Nan')
            raise Warning('Passed list is empty')
        elif type(list_to_write[0]) == tuple:
            for item in sort_slice(list_to_write, n):
                file.write(item[0] + '\n')
        elif type(list_to_write[0]) == str:
            for item in list_to_write:
                file.write(item + '\n')
        else:
            raise TypeError(
                f'incorrect type, list[str] and list[tuple(str, float)] supported only,'
                f' but {type(list_to_write[0])} was passed')


write_results(top_100_cpgs_ktest, 100, 'ktest')
write_results(top_200_cpgs_ktest, 200, 'ktest')
write_results(top_300_cpgs_ktest, 300, 'ktest')

### Отбор признаков с помощью `sklearn.SelectKBest` и функции отбора `f_classif`.
#### Отбор `100`, `200`, `300` признаков для дальнейшего обучения модели.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

df_x = df_x.drop('status', axis=1)


def select_n_best(x, y, n):
    selector_k = SelectKBest(score_func=f_classif, k=n)
    selector_k.fit(x, y)
    cols = selector_k.get_support(indices=True)
    return cols

In [ ]:
top_100_cpgs_skb = list(df_x.columns[select_n_best(df_x, df_y, 100)])
top_200_cpgs_skb = list(df_x.columns[select_n_best(df_x, df_y, 200)])
top_300_cpgs_skb = list(df_x.columns[select_n_best(df_x, df_y, 300)])

In [ ]:
write_results(top_100_cpgs_skb, 100, 'skb')
write_results(top_200_cpgs_skb, 200, 'skb')
write_results(top_300_cpgs_skb, 300, 'skb')

### Пересечение списков отобранных cpg с помощью вышеуказанных методов

In [ ]:
def intersection(ktest, skb):
    k_cpgs = []
    for item in ktest:
        k_cpgs.append(item[0])

    common_cpgs = [cpg for cpg in skb if cpg in k_cpgs]
    return common_cpgs

In [ ]:
write_results(intersection(top_100_cpgs_ktest, top_100_cpgs_skb), 100, 'intersection')
write_results(intersection(top_100_cpgs_ktest, top_100_cpgs_skb), 200, 'intersection')
write_results(intersection(top_100_cpgs_ktest, top_100_cpgs_skb), 300, 'intersection')